# Testing the performance of VGG16

In [1]:
# import cv2
# import os

# input_folder = "./data"
# output_folder = "./data"

# os.makedirs(output_folder, exist_ok=True)  # Ensure output directory exists

# for filename in os.listdir(input_folder):
#     if filename.endswith(".png"):  
#         img_path = os.path.join(input_folder, filename)
        
#         # Read the image (forcefully add an alpha channel if missing)
#         img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        
#         if img.shape[2] == 3:  # If no alpha channel, add one
#             b, g, r = cv2.split(img)
#             a = np.ones_like(b) * 255  # Create a full white alpha channel
#             img = cv2.merge((b, g, r, a))

#         # Save image
#         cv2.imwrite(os.path.join(output_folder, filename), img)
#         print(f"Converted {filename} to RGBA.")

In [2]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow import keras
import tensorflow as tf
import os
import numpy as np

2025-02-25 12:52:49.734259: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-25 12:52:49.756742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740484369.771097   87946 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740484369.774707   87946 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 12:52:49.791617: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
tf.config.set_visible_devices([], 'GPU')

In [14]:
def image_files(path):
    '''
    Creates a list with the names of the images to be processed.
    '''
    image_files = []
    # creates a ScandirIterator aliased as files
    with os.scandir(path) as files:
        for file in files:
            if file.name.endswith('.jpg'):
                # adds only the image files to the list
                image_files.append(file.name)
    return sorted(image_files)


def load_model():
    model = VGG16()
    model = Model(inputs = model.inputs, outputs = model.output)
    return model


def recognize_objects(file, path, model):
    '''
    Recognizes objects in an image file using the VGG16 model.
    '''
    # load the image as a 224x224 array
    img = load_img(f'{path}/{file}', target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img, dtype=np.float32)
    
    reshaped_img = img.reshape(1,224,224,3)
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    objects = model.predict(imgx)

    return objects

In [15]:
path = "./data"
model = load_model()
#print(model.input_shape)  # Expected input shape

files = image_files(path)

for file in files: 
    preds= recognize_objects(file, path, model)
    top_preds = decode_predictions(preds, top=5)[0]

    print(f"Objects detected in {file} are:")
    for i, (imagenet_id, label, score) in enumerate(top_preds):
        print(f"{i+1}. {label} ({score*100:.2f}%)")
    print()
       

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step

/home/rallypal/.pyenv/versions/lewagon/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_115']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
Objects detected in 01.jpg are:
1. crutch (7.83%)
2. swab (7.47%)
3. drum (6.01%)
4. panpipe (5.55%)
5. sombrero (3.34%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Objects detected in 02.jpg are:
1. fountain (15.15%)
2. swab (6.54%)
3. crutch (6.14%)
4. umbrella (5.21%)
5. trench_coat (4.38%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Objects detected in 03.jpg are:
1. trench_coat (9.80%)
2. hair_spray (9.43%)
3. abaya (8.95%)
4. academic_gown (7.39%)
5. cleaver (7.20%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Objects detected in 04.jpg are:
1. library (22.30%)
2. gondola (19.80%)
3. bookshop (18.81%)
4. streetcar (16.26%)
5. prison (2.46%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Objects detected in 05.jpg are:
1. obelisk (16.47%)
2. church (9.61%)
3. beacon (9.59%)
4. flagpole (8.93%)
5. megalith (7.47%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Objects detected in 06.jpg are:
1. limousine (81.23%)
2. groom (5.36%)
3. miniskirt (4.24%)
4. bullet_train (1.

# Testing the performance of CLIP:

In [30]:
import os
import torch
import clip
from PIL import Image
from pathlib import Path

# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

prompt= 'a smiling woman'
text = clip.tokenize([prompt]).to(device)

# Path to the image directory
image_dir = Path("./data")

my_images = []

for image_path in sorted(image_dir.glob("*.jpg")):  
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    # Compute image features
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

    # Compute similarity score between the image and the "car" prompt
    similarity = image_features @ text_features.T

    # The threshold is the key!
    threshold = 23.0  
    if similarity.item() > threshold:
        my_images.append(image_path)

print(f"Images that display {prompt}:")
for each_image in my_images:
    print(each_image)

Images that display a smiling woman:
data/02.jpg
data/03.jpg
data/09.jpg
data/10.jpg
